In [3]:
import numpy as np
import pandas as pd
import xlsxwriter

In [4]:
T= pd.read_excel("../DataFiles/Traffic to Contract History.xlsx")
dbCH= pd.read_excel("../DataFiles/Contract History.xlsx")
dbCO= pd.read_excel("../DataFiles/Contracts.xlsx")
SU = pd.read_excel("../DataFiles/Survey Units.xlsx")
SDD = pd.read_excel("../DataFiles/Survey Distress Details.xlsx")

In [5]:
#Selecting the required columns from the datasets
dbT = T[['Traffic Year', 'Traffic Year ESALs _One Direction_', 'Contract History ID']]
dbT.columns = ['Traffic Year', 'ESALs', 'Contract History ID'] #remaning the column titles for traffic df.
dbCH = dbCH[['SR','Begin ARM', 'End ARM', 'Region', 'ID', 'Construction Year', 'Major Rehab Year', 'Segment Lane Miles', 
            'ESALs Since Major Rehab', 'Total Thickness', 'Total Surface Thickness', 'Listed Surface Type', 'Pavement Type',
            'Construction Pavement Type', 'Major Rehab Type', 'Prior Major Rehab Year', 'Prior Major Rehab Type']]
dbC = dbCO[['SR', 'Begin ARM', 'End ARM', 'ID','Construction End Date','Substantial Completion Date','Contract Type Description', 'Surface Type Code', 'Surface Type Description', 'Surface Thickness',
          'Treat Base Type Code', 'Treat Base Type Description', 'Treat Base Thickness', 'UTreat Base Thickness']]

In [6]:
#creating the 'Year' variable in contract history df.
def getYear(year):
    return int(str(year).split('-')[0])

In [7]:
yearVec = dbC['Construction End Date'].apply(getYear)

In [8]:
dbC.loc[:,'Year'] = yearVec

/home/guerrero/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:249: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/guerrero/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
# dbC.head()

In [10]:
#Selecting only segments which have had a resurfacing activity applied
SurfType = [u'FIRST STAGE RESURFACING', u'RESURFACE EXISTING RDWY']
dbCs = dbC[dbC.loc[:,'Contract Type Description'].isin(SurfType)]

In [11]:
print len(dbCs)

31130


In [12]:
db = dbCs[['SR', 'Begin ARM', 'End ARM', 'ID','Year','Contract Type Description', 'Surface Type Code', 
           'Surface Type Description', 'Surface Thickness']]

Merging the Contracts and Contracts-History Datasets

In [13]:
CCH = pd.merge(db, dbCH, how = 'left',left_on = ['SR', 'Begin ARM', 'End ARM'], 
                right_on = ['SR', 'Begin ARM', 'End ARM'])

In [14]:
CCHs = CCH.dropna(axis=0, how='any', thresh=None, subset=['Construction Year'], inplace=False)

In [15]:
print len(CCHs), len(CCH)

28706 49298


In [18]:
SU.columns

Index([                      u'SR',                      u'RRT',
                            u'RRQ',                u'Begin ARM',
                        u'End ARM',               u'Begin SRMP',
                       u'End SRMP',   u'Direction To Inventory',
                    u'Original ID',                 u'Due Year',
               u'Due Year Trigger',                u'Is Bridge',
                 u'Is Near Tunnel',      u'Is Near RR Crossing',
              u'Lane Surface Type',               u'Lane Miles',
                         u'Region',                      u'NHS',
       u'Federal Functional Class',    u'Avg Construction Date',
           u'Avg Major Rehab Date',                   u'Int ID'],
      dtype='object')

Merging dabase to Survey Units dataset

In [19]:
SUCCH = pd.merge(CCHs, SU, how = 'left',left_on = ['SR', 'Begin ARM', 'End ARM', 'Region', 'Segment Lane Miles'],
               right_on = ['SR', 'Begin ARM', 'End ARM', 'Region', 'Lane Miles'])

In [20]:
#removing unique segments not contained in both datasets
db2 = SUCCH.dropna(axis=0, how='any', thresh=None, subset=['Lane Miles'], inplace=False)

In [21]:
print len(SU)
print len(db2)
print len(db2[db2.loc[:,'Year']>1980])
print len(np.unique(np.array(db2[db2.loc[:,'Year']>1980].loc[:, 'Int ID'])))
print len(np.unique(np.array(db2[db2.loc[:,'Year']>1980].loc[:, [u'SR', u'Begin ARM', u'End ARM']])))

91022
7351
6623
1799
2506


Adding the Survey Details dataset including the cracking variables

In [22]:
sdd = SDD[[u'Year', u'Low Alligator Cracking', u'Medium Alligator Cracking',u'High Alligator Cracking',
          u'Low Flexible Longitudinal Cracking',u'Medium Flexible Longitudinal Cracking', u'High Flexible Longitudinal Cracking',u'Sealed Alligator',
           u'Sealed Longitudinal', u'Bridge', u'Int ID']]
sdd.columns = [u'YearC', u'Low Alligator Cracking', u'Medium Alligator Cracking',u'High Alligator Cracking',
          u'Low Flexible Longitudinal Cracking',u'Medium Flexible Longitudinal Cracking', u'High Flexible Longitudinal Cracking',u'Sealed Alligator',
           u'Sealed Longitudinal', u'Bridge', u'Int ID']

Merging dabase to Survey-Units-Details dataset


In [23]:
SCV = pd.merge(sdd, db2, how = 'left',left_on = ['Int ID'], right_on = ['Int ID'])

In [24]:
db3 = SCV.dropna(axis=0, how='any', thresh=None, subset=['Lane Miles'], inplace=False)

In [25]:
print len(db3)
print len(np.unique(np.array(db3.loc[:, [u'SR', u'Begin ARM', u'End ARM']])))

87195
1555


Merging the cracking distress variables

In [26]:
#sc is the dataset with selected vars necessary for formulating the crack initiation model
sc = db3[[u'YearC', u'Low Alligator Cracking', u'Medium Alligator Cracking', u'High Alligator Cracking',
          u'Low Flexible Longitudinal Cracking', u'Medium Flexible Longitudinal Cracking', u'High Flexible Longitudinal Cracking',u'Sealed Alligator',
          u'Sealed Longitudinal', u'Int ID', u'SR', u'Begin ARM', u'End ARM', u'Year', u'Contract Type Description',
          u'Surface Type Code', u'Surface Type Description', u'Surface Thickness', u'Region', u'ID_y', u'Construction Year',
          u'Major Rehab Year', u'Segment Lane Miles', u'ESALs Since Major Rehab', u'Total Thickness', u'Total Surface Thickness',
          u'Pavement Type', u'Construction Pavement Type', u'Major Rehab Type',u'Prior Major Rehab Year',
          u'Prior Major Rehab Type', u'Original ID', u'Is Bridge', u'Lane Surface Type', u'Lane Miles',
                    u'Federal Functional Class',  u'Avg Construction Date',   u'Avg Major Rehab Date']]

In [27]:
sc.loc[:,'Alli'] = 'nan'
sc.loc[:,'CatAlli'] = 'nan'
sc.loc[:,"Long"] = 'nan'
sc.loc[:,"CatLong"] = 'nan'

In [28]:
import math
def getAlli(row):
    p1 = row[u'Low Alligator Cracking']
    p2 = row[u'Medium Alligator Cracking']
    p3 = row[u'High Alligator Cracking']
    
    if not math.isnan(p1):
        a = p1
    elif not math.isnan(p2):
        a = p2
    elif not math.isnan(p3):
        a = p3
        at = 'H'
    else:
        a = 'nan'
    return a

In [29]:
def getCatAlli(row):
    p1 = row[u'Low Alligator Cracking']
    p2 = row[u'Medium Alligator Cracking']
    p3 = row[u'High Alligator Cracking']
    
    if not math.isnan(p1):
        at = 'L'
    elif not math.isnan(p2):
        at = 'M'
    elif not math.isnan(p3):
        at = 'H'
    else:
        at = 'nan'
    return at

In [30]:
def getLong(row):
    p4 = row[u'Low Flexible Longitudinal Cracking']
    p5 = row[u'Medium Flexible Longitudinal Cracking']
    p6 = row[u'High Flexible Longitudinal Cracking'] 
        
    if not math.isnan(p4):
        l = p4
    elif not math.isnan(p5):
        l = p5
    elif not math.isnan(p6):
        l = p6
    else:
        l = 'nan'
    return l

In [31]:
def getCatLong(row):
    p4 = row[u'Low Flexible Longitudinal Cracking']
    p5 = row[u'Medium Flexible Longitudinal Cracking']
    p6 = row[u'High Flexible Longitudinal Cracking'] 
        
    if not math.isnan(p4):
        lt = 'L'
    elif not math.isnan(p5):
        lt = 'M'
    elif not math.isnan(p6):
        lt = 'H'
    else:
        lt = 'nan'
    return lt

In [32]:
sc.loc[:,'Alli'] = sc.apply(getAlli, axis =1)
sc.loc[:,'CatAlli'] = sc.apply(getCatAlli, axis =1)
sc.loc[:,"Long"] = sc.apply(getLong, axis =1)
sc.loc[:,"CatLong"] = sc.apply(getCatLong, axis =1)

In [33]:
cdb = sc[~((sc.loc[:,'Alli']== 'nan') & (sc.loc[:,'Long']== 'nan'))]

In [34]:
len(cdb)

58321

In [35]:
df = cdb[[u'YearC',u'Sealed Alligator', u'Sealed Longitudinal', u'Int ID', u'SR', u'Begin ARM', u'End ARM', u'Year', 
         u'Contract Type Description', u'Surface Type Code', u'Surface Type Description', u'Surface Thickness', 
         u'Region', u'ID_y', u'Construction Year', u'Major Rehab Year', u'Segment Lane Miles', u'ESALs Since Major Rehab',
         u'Total Thickness', u'Total Surface Thickness', u'Pavement Type', u'Construction Pavement Type', 
         u'Major Rehab Type',u'Prior Major Rehab Year', u'Prior Major Rehab Type', u'Original ID', 
         u'Is Bridge', u'Lane Surface Type', u'Lane Miles', u'Federal Functional Class',  u'Avg Construction Date', 
         u'Avg Major Rehab Date', 'Alli', 'CatAlli', 'Long', 'CatLong']]

In [40]:
def change2Percentage(row):
    p = row[u'Alli']
    m = row[u'Lane Miles']
    if not (p == 'nan'):
        return p/(m*5280) * 100
    else:
        return 'nan'  

In [41]:
def change2PercentageL(row):
    p = row[u'Long']
    m = row[u'Lane Miles']
    if not (p == 'nan'):
        return p/(m*5280) * 100
    else:
        return 'nan' 

In [42]:
df.loc[:, 'AlliP'] = df.apply(change2Percentage, axis = 1)
df.loc[:, 'LongP'] = df.apply(change2PercentageL, axis = 1)

Merging the Traffic dataset

In [45]:
db4 = pd.merge(df, dbT, how = 'left',left_on = ['ID_y'], right_on = ['Contract History ID'])

In [46]:
df2 = db4.dropna(axis=0, how='any', thresh=None, subset=[u'Traffic Year', u'ESALs'], inplace=False)

In [47]:
print len(df2)
print len(np.unique(np.array(df2.loc[:, [u'Contract History ID']])))

52899
972


In [48]:
MatType = np.unique(np.array(df2.loc[:, 'Surface Type Code']))
dfst = df2.drop_duplicates(['Contract History ID'])

/home/guerrero/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:198: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [49]:
print len(MatType)
print len(dfst)

70
972


In [60]:
MTdic = {}
for mt in MatType[1:]:
    n = len(dfst[dfst.loc[:, 'Surface Type Code']==mt])
    if n > 30:
        MTdic[mt] = n

In [61]:
print len(MTdic)

11


In [62]:
print MTdic

{u'AA': 34, u'G5': 33, u'BA': 216, u'AI': 34, u'PB': 34, u'Z4': 32, u'A2': 41, u'A5': 37, u'B2': 124, u'X2': 35, u'G2': 60}


In [63]:
def catSurfType(row):
    st = row['Surface Type Code']
    if st in MTdic.keys():
        return st
    else:
        return 'Other'  

In [64]:
df2["STcode"] = df2.apply(catSurfType, axis = 1)

/home/guerrero/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [65]:
df2.head()

,YearC,Sealed Alligator,Sealed Longitudinal,Int ID,SR,Begin ARM,End ARM,Year,Contract Type Description,Surface Type Code,...,Alli,CatAlli,Long,CatLong,AlliP,LongP,Traffic Year,ESALs,Contract History ID,STcode
0,2003,NaN,NaN,54964,101,0.53,0.63,1996,RESURFACE EXISTING RDWY,AA,...,21,L,10,L,1.988636,0.9469697,2013,32740,0073c3c3-1b5a-40af-a002-022c09a2875d,AA
1,2004,NaN,NaN,54964,101,0.53,0.63,1996,RESURFACE EXISTING RDWY,AA,...,10,L,84,L,0.9469697,7.954545,2013,32740,0073c3c3-1b5a-40af-a002-022c09a2875d,AA
2,2005,NaN,NaN,54964,101,0.53,0.63,1996,RESURFACE EXISTING RDWY,AA,...,21,L,10,L,1.988636,0.9469697,2013,32740,0073c3c3-1b5a-40af-a002-022c09a2875d,AA
3,2006,NaN,NaN,54964,101,0.53,0.63,1996,RESURFACE EXISTING RDWY,AA,...,21,L,5,L,1.988636,0.4734848,2013,32740,0073c3c3-1b5a-40af-a002-022c09a2875d,AA
4,2007,NaN,NaN,54964,101,0.53,0.63,1996,RESURFACE EXISTING RDWY,AA,...,nan,nan,5,L,nan,0.4734848,2013,32740,0073c3c3-1b5a-40af-a002-022c09a2875d,AA


In [66]:
df2.to_excel("../DataFiles/RcrackModeldataset.xlsx")